# **Anime Recommender System on Markdown Notebook**



## Introduction

This notebook outlines the steps taken to build a collaborative and content-based recommender system for a collection of anime titles. The goal is to predict how a user will rate an anime title they have not yet viewed, based on their historical preferences and the characteristics of the anime.


## Project Motivation and Importance

In the age of digital content, recommender systems have become essential tools for personalized content delivery. These systems enhance user experience by suggesting relevant items based on user behavior and item attributes. For anime enthusiasts, discovering new shows that match their tastes can be a delightful experience, and an efficient recommender system can significantly enhance user satisfaction and engagement. 

Building an anime recommender system can:

1. **Improve User Experience**: By suggesting anime titles that align with users' preferences, the system can help users find content they are likely to enjoy, reducing the time spent searching for new shows.
2. **Increase Engagement**: Personalized recommendations can increase user interaction and retention, as users are more likely to return to the platform that consistently provides content they like.
3. **Drive Business Value**: For platforms hosting anime content, effective recommendation systems can lead to higher user satisfaction, more views, and ultimately, increased revenue.
4. **Learn User Preferences**: By analyzing user ratings and behavior, the system can gain insights into trends and preferences, which can be valuable for content creators and distributors.

## Steps Covered:
1. Data Loading 
2. Preprocessing
3. Exploratory Data Analysis (EDA)
4. Model Training
5. Model Evaluation
6. Conclusion


## 1. Data Loading and Preprocessing

### Importing Necessary Packages


In [76]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel


### Loading the dataset

In [77]:
# Load the dataset
anime_data = pd.read_csv('anime.csv')
submission_data=pd.read_csv ('submission.csv')
test_data=pd.read_csv ('test.csv')
train_data=pd.read_csv ('train.csv')

In [78]:

# Display the first few rows of the anime dataset
train_data.head()

,user_id,anime_id,rating
0,1,11617,10
1,1,11757,10
2,1,15451,10
3,2,11771,10
4,3,20,8


### Making Copies of Original Datasets**

To minimize the risk of accidental data loss or irreversible changes during the cleaning operations and process, we create copies of the original datasets.This precaution ensures that the original data remains unchanged and can be referred back to if necessary.

In [80]:
train_data = train_data.copy(deep=True)
test_data = test_data.copy(deep=True)
anime_data = test_data.copy(deep=True)
submission_data_df = test_data.copy(deep=True)

##  2. Data Cleaning and Preprocessing

## 3. EDA

## 4. Model Training

## 5. Model Evaluation


## 6. Conclusion